In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

In [2]:
df_potency_prediction = pd.DataFrame()

SARS potency

In [3]:
data = pd.read_csv('featurized_data.csv').drop(columns=['pIC50 (MERS-CoV Mpro)', 'my_id'])
data

,SMILES,pIC50 (SARS-CoV-2 Mpro),training_set,0,1,2,3,4,5,6,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,COC[C@]1(C)C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)N1C...,NaN,True,1.675444,-0.944227,-1.251482,-1.398225,-1.705958,0.017005,-2.614705,...,0.474973,1.807153,-1.547823,0.223986,-0.051001,0.435159,0.584132,0.395009,-0.182329,0.700669
1,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,5.29,True,-0.683850,-1.172282,1.291860,-1.441717,0.974246,0.604147,-5.258241,...,1.144030,1.240560,-0.778391,0.974543,0.389707,-0.136449,0.215756,1.155893,-0.744604,0.048368
2,CNC(=O)CN1C[C@]2(C[C@H](C)N(C3=CN=CC=C3C3CC3)C...,NaN,True,-1.063032,2.085217,3.236487,-2.473934,1.469923,-2.766837,-7.845398,...,0.748969,1.131117,-0.938964,0.496262,-0.313748,-0.248265,-0.427747,-0.501517,0.300073,0.529345
3,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,6.11,True,-1.538650,-1.734368,2.749866,-1.587451,0.483516,0.321441,-5.313765,...,1.188415,1.692552,-0.712264,0.971305,0.282944,-0.099405,-0.053377,0.915527,-0.426292,-0.395897
4,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,5.62,True,-0.760658,-1.610037,2.838731,-1.661773,0.356773,0.506010,-4.773226,...,1.345026,1.788778,-0.557263,1.257126,0.231708,0.022518,0.374909,1.158598,-0.543888,-0.145062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...,NaN,False,-1.805614,4.241495,0.263991,-0.479579,1.350500,-2.525100,-1.938316,...,1.236730,1.782411,-0.063976,1.842579,0.371753,1.061931,-0.114359,0.582579,-0.564786,0.819932
1324,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...,NaN,False,-1.805614,4.241495,0.263991,-0.479579,1.350500,-2.525100,-1.938316,...,1.236730,1.782411,-0.063976,1.842579,0.371753,1.061931,-0.114359,0.582579,-0.564786,0.819932
1325,O=C(CC1=CN=CC2=CC=CC=C12)N1CCC[C@H]2CCCC[C@@H]...,NaN,False,0.529626,3.191018,-1.929221,-0.021572,2.940725,-2.550586,-1.202677,...,0.895240,1.630075,-0.043119,2.017280,0.443292,0.771463,-0.079971,1.343421,-0.985570,1.089759
1326,COC1=CC=CC=C1[C@H]1C[C@H](C)CCN1C(=O)CC1=CN=CC...,NaN,False,1.765704,0.740447,1.678681,-0.759331,-0.727229,-3.069711,-2.005487,...,1.031603,1.161544,0.212015,1.766024,1.087408,1.016478,0.199734,1.628376,-0.881626,0.827574


In [4]:
train_data = data[(data.training_set) & (~data['pIC50 (SARS-CoV-2 Mpro)'].isna())].reset_index(drop=True).drop(columns=['training_set'])
test_data = data[~data.training_set].reset_index(drop=True).drop(columns=['training_set']).drop(columns='pIC50 (SARS-CoV-2 Mpro)')

In [5]:
train_data

,SMILES,pIC50 (SARS-CoV-2 Mpro),0,1,2,3,4,5,6,7,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,5.29,-0.683850,-1.172282,1.291860,-1.441717,0.974246,0.604147,-5.258241,1.750730,...,1.144030,1.240560,-0.778391,0.974543,0.389707,-0.136449,0.215756,1.155893,-0.744604,0.048368
1,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,6.11,-1.538650,-1.734368,2.749866,-1.587451,0.483516,0.321441,-5.313765,2.188489,...,1.188415,1.692552,-0.712264,0.971305,0.282944,-0.099405,-0.053377,0.915527,-0.426292,-0.395897
2,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,5.62,-0.760658,-1.610037,2.838731,-1.661773,0.356773,0.506010,-4.773226,2.562765,...,1.345026,1.788778,-0.557263,1.257126,0.231708,0.022518,0.374909,1.158598,-0.543888,-0.145062
3,C=C(CN(C(=O)C1CCOC2=C1C=C(Cl)C=C2)C1=CN=CC2=CC...,6.45,-0.717072,4.248509,2.276852,0.884355,3.553937,-2.319796,-2.773176,0.065285,...,1.271122,1.666859,-0.125508,0.987690,0.243234,-0.063366,-0.047070,1.528865,-0.191177,-0.086432
4,C=C(CN(C(=O)C1CCOC2=C1C=C(Cl)C=C2)C1=CN=CC2=CC...,5.56,-0.110807,4.685553,2.251827,1.852081,3.770436,-3.042558,-2.969104,0.124858,...,1.451560,1.757837,-0.501121,0.891049,0.144301,-0.068077,0.032858,1.126181,0.014145,-0.472433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,O=C(CC1=CN=CC2=CC=CC=C12)N1CCC([C@H]2CCOC2)CC1...,4.68,0.311992,0.681946,1.497754,1.085060,3.043362,-1.524963,1.506631,-2.062687,...,1.225418,1.560984,0.212282,2.242753,0.277963,0.634615,0.922421,1.101674,-1.212864,0.942894
838,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC2(CC2)C1,4.41,1.138839,2.368807,-0.913875,0.422662,2.809446,-2.669876,-0.685522,-0.268157,...,1.149384,1.677205,-0.233706,1.950044,0.162290,0.740121,0.632765,1.244563,-1.248425,1.474069
839,CNS(=O)(=O)OCC(=O)N1CCN(CC2=CC=CC(Cl)=C2)[C@H]...,6.38,0.978807,0.452567,3.578018,-1.500762,-3.953382,3.328140,-1.607548,0.510762,...,0.207879,1.332996,-0.675637,-1.112114,-0.189924,-0.633633,0.487192,-0.357850,-0.978905,0.237751
840,O=C(CC1=CN=CC2=CC=CC=C12)N1CC[C@@H]2CCCC[C@H]2...,6.09,1.506480,2.357378,-0.835635,0.813551,3.241050,-2.768027,-0.532689,0.384822,...,0.931776,1.438686,0.144391,2.237076,0.298610,0.709513,0.463113,1.451820,-1.149531,1.244195


In [6]:
test_data

,SMILES,0,1,2,3,4,5,6,7,8,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,C=CC(=O)NC1=CC=CC(N(CC2=CC=CC(Cl)=C2)C(=O)CC2=...,-0.066101,0.514035,2.406722,-0.538694,2.758708,-1.015454,-3.046322,0.896387,-4.450608,...,1.007086,1.268912,-0.115565,0.872421,0.712432,0.124430,0.561479,1.282628,-0.921412,-0.077929
1,CNC(=O)CN1C[C@@]2(C(=O)N(C3=CN=CC4=CC=CC=C34)C...,0.709517,2.181829,2.672621,-2.260278,1.534939,-1.853767,-7.244531,1.715286,-5.248160,...,0.514781,1.563557,-0.539677,0.904415,-0.354920,1.015560,-0.088539,0.839184,-0.296548,1.197845
2,CNC(=O)CN1C[C@]2(CCN(C3=CN=CC4=CC=C(OC[C@H](O)...,-0.252611,1.193684,3.419403,-2.633735,1.034602,-3.068882,-6.420348,-0.251201,-4.888867,...,0.873598,1.998666,-0.840693,1.153143,-0.296099,-0.307304,-0.072586,0.109765,-0.695365,0.420144
3,CNC(=O)CN1C[C@@]2(C(=O)N(C3=CN=CC4=CC=CC=C34)C...,0.492067,2.421684,1.147309,-2.394543,1.914426,-1.966078,-7.571744,1.693252,-5.183313,...,0.769476,1.147141,-0.487252,0.878944,-0.238295,0.971715,0.266798,1.186951,-0.876289,1.392308
4,CNC(=O)CN1C[C@@]2(C(=O)N(C3=CN=CC4=CC=CC=C34)C...,0.068854,2.535121,2.766077,-2.350221,0.772496,-1.514147,-7.961858,2.450783,-4.384021,...,0.919464,1.261334,-0.752012,0.995637,-0.262354,0.107522,0.189838,0.584094,-0.818777,0.789732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...,-1.805614,4.241495,0.263991,-0.479579,1.350500,-2.525100,-1.938316,-0.564988,0.641694,...,1.236730,1.782411,-0.063976,1.842579,0.371753,1.061931,-0.114359,0.582579,-0.564786,0.819932
293,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...,-1.805614,4.241495,0.263991,-0.479579,1.350500,-2.525100,-1.938316,-0.564988,0.641694,...,1.236730,1.782411,-0.063976,1.842579,0.371753,1.061931,-0.114359,0.582579,-0.564786,0.819932
294,O=C(CC1=CN=CC2=CC=CC=C12)N1CCC[C@H]2CCCC[C@@H]...,0.529626,3.191018,-1.929221,-0.021572,2.940725,-2.550586,-1.202677,-1.666443,-2.110840,...,0.895240,1.630075,-0.043119,2.017280,0.443292,0.771463,-0.079971,1.343421,-0.985570,1.089759
295,COC1=CC=CC=C1[C@H]1C[C@H](C)CCN1C(=O)CC1=CN=CC...,1.765704,0.740447,1.678681,-0.759331,-0.727229,-3.069711,-2.005487,-3.908790,-1.688375,...,1.031603,1.161544,0.212015,1.766024,1.087408,1.016478,0.199734,1.628376,-0.881626,0.827574


In [7]:
task = 'pIC50 (SARS-CoV-2 Mpro)'
kernel = 'rbf'
C = 2
epsilon = 0.02
seed = 1

train_data = train_data.set_index('SMILES', drop=True)
train_data = shuffle(train_data, random_state=seed)
X_train = train_data.drop(columns=task)
y_train = train_data[task]

test_data = test_data.set_index('SMILES', drop=True)
X_test = test_data

current_model = SVR(kernel=kernel, C=C, epsilon=epsilon)
current = make_pipeline(MinMaxScaler(), current_model)
current.fit(X_train,y_train)
y_pred = current.predict(X_test)

In [8]:
y_pred

array([5.14478785, 6.28169104, 6.91386566, 5.38482387, 6.304187  ,
       6.24802636, 6.81902595, 6.75781389, 6.39622908, 6.82361453,
       5.28265495, 6.47915649, 6.68990879, 6.81076223, 6.81554709,
       6.44205827, 6.23877754, 6.60264492, 5.3297664 , 6.69847216,
       6.59317272, 7.0343077 , 6.54182801, 7.63241082, 6.40221269,
       6.91008833, 6.50924665, 6.97569407, 6.52412271, 5.63034986,
       6.7210281 , 7.12668754, 6.22672993, 6.62666269, 6.52438952,
       6.53062479, 7.06127098, 7.02064473, 6.67684148, 6.87480238,
       6.34709386, 6.41883447, 6.73936626, 6.73936626, 6.6501776 ,
       6.6501776 , 6.26050716, 6.26050716, 6.2226951 , 6.2226951 ,
       6.71593877, 6.56090643, 6.69681218, 6.26767057, 6.67119816,
       6.26125067, 6.72555789, 7.32527163, 6.76153266, 6.36894319,
       6.52907289, 6.65834632, 6.01956411, 6.60065699, 7.22069126,
       7.08838516, 7.51688692, 7.45699402, 7.08854817, 6.35246005,
       6.50125045, 6.9848965 , 7.40729877, 6.23315974, 6.50579

In [9]:
df_potency_prediction['SARS'] = y_pred
df_potency_prediction

,SARS
0,5.144788
1,6.281691
2,6.913866
3,5.384824
4,6.304187
...,...
292,4.246918
293,4.246918
294,4.658370
295,5.923428


MERS potency

In [10]:
data = pd.read_csv('featurized_data.csv').drop(columns=['pIC50 (SARS-CoV-2 Mpro)', 'my_id'])
data

,SMILES,pIC50 (MERS-CoV Mpro),training_set,0,1,2,3,4,5,6,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,COC[C@]1(C)C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)N1C...,4.19,True,1.675444,-0.944227,-1.251482,-1.398225,-1.705958,0.017005,-2.614705,...,0.474973,1.807153,-1.547823,0.223986,-0.051001,0.435159,0.584132,0.395009,-0.182329,0.700669
1,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,4.92,True,-0.683850,-1.172282,1.291860,-1.441717,0.974246,0.604147,-5.258241,...,1.144030,1.240560,-0.778391,0.974543,0.389707,-0.136449,0.215756,1.155893,-0.744604,0.048368
2,CNC(=O)CN1C[C@]2(C[C@H](C)N(C3=CN=CC=C3C3CC3)C...,4.73,True,-1.063032,2.085217,3.236487,-2.473934,1.469923,-2.766837,-7.845398,...,0.748969,1.131117,-0.938964,0.496262,-0.313748,-0.248265,-0.427747,-0.501517,0.300073,0.529345
3,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,4.90,True,-1.538650,-1.734368,2.749866,-1.587451,0.483516,0.321441,-5.313765,...,1.188415,1.692552,-0.712264,0.971305,0.282944,-0.099405,-0.053377,0.915527,-0.426292,-0.395897
4,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,4.81,True,-0.760658,-1.610037,2.838731,-1.661773,0.356773,0.506010,-4.773226,...,1.345026,1.788778,-0.557263,1.257126,0.231708,0.022518,0.374909,1.158598,-0.543888,-0.145062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...,NaN,False,-1.805614,4.241495,0.263991,-0.479579,1.350500,-2.525100,-1.938316,...,1.236730,1.782411,-0.063976,1.842579,0.371753,1.061931,-0.114359,0.582579,-0.564786,0.819932
1324,O=C(CC1=CN=CC2=CC=CC=C12)N1CCCC[C@H]1[C@H]1CCC...,NaN,False,-1.805614,4.241495,0.263991,-0.479579,1.350500,-2.525100,-1.938316,...,1.236730,1.782411,-0.063976,1.842579,0.371753,1.061931,-0.114359,0.582579,-0.564786,0.819932
1325,O=C(CC1=CN=CC2=CC=CC=C12)N1CCC[C@H]2CCCC[C@@H]...,NaN,False,0.529626,3.191018,-1.929221,-0.021572,2.940725,-2.550586,-1.202677,...,0.895240,1.630075,-0.043119,2.017280,0.443292,0.771463,-0.079971,1.343421,-0.985570,1.089759
1326,COC1=CC=CC=C1[C@H]1C[C@H](C)CCN1C(=O)CC1=CN=CC...,NaN,False,1.765704,0.740447,1.678681,-0.759331,-0.727229,-3.069711,-2.005487,...,1.031603,1.161544,0.212015,1.766024,1.087408,1.016478,0.199734,1.628376,-0.881626,0.827574


In [11]:
train_data = data[(data.training_set) & (~data['pIC50 (MERS-CoV Mpro)'].isna())].reset_index(drop=True).drop(columns=['training_set'])
test_data = data[~data.training_set].reset_index(drop=True).drop(columns=['training_set']).drop(columns='pIC50 (MERS-CoV Mpro)')

In [12]:
task = 'pIC50 (MERS-CoV Mpro)'
kernel = 'rbf'
C = 0.8
epsilon = 0.0001
seed = 1

train_data = train_data.set_index('SMILES', drop=True)
train_data = shuffle(train_data, random_state=seed)
X_train = train_data.drop(columns=task)
y_train = train_data[task]

test_data = test_data.set_index('SMILES', drop=True)
X_test = test_data

current_model = SVR(kernel=kernel, C=C, epsilon=epsilon)
current = make_pipeline(MinMaxScaler(), current_model)
current.fit(X_train,y_train)
y_pred = current.predict(X_test)

In [13]:
y_pred

array([5.3239538 , 5.1238921 , 5.31347329, 4.64920025, 5.29278616,
       5.18800819, 5.8330839 , 5.85268807, 5.3975494 , 5.41269273,
       4.59484705, 5.43468101, 5.44010369, 5.45877131, 5.65200588,
       5.22730066, 5.20628358, 5.37106534, 4.32056578, 5.04299827,
       5.33064148, 5.71355732, 5.45778128, 6.16251197, 5.17966503,
       5.67657963, 5.37415066, 5.69546172, 5.36166634, 5.5763098 ,
       5.56916494, 5.84463718, 5.4624686 , 5.50076812, 5.49071915,
       5.50665814, 5.95577294, 5.85535017, 5.05874266, 4.99316086,
       5.39884664, 5.51428123, 5.69577681, 5.69577681, 5.72323697,
       5.72323697, 5.55161176, 5.55161176, 5.52843179, 5.52843179,
       5.54727797, 5.4719747 , 5.31232429, 5.42107001, 5.52363032,
       5.39346159, 5.56927468, 6.08730464, 5.15407361, 5.5024087 ,
       5.29162842, 5.52683984, 5.34553909, 5.62629857, 5.97423728,
       5.85726692, 5.87371175, 6.50376288, 5.94447448, 5.35436787,
       5.59325234, 5.76642927, 6.03081465, 5.46861071, 5.57474

In [14]:
df_potency_prediction['MERS'] = y_pred
df_potency_prediction

,SARS,MERS
0,5.144788,5.323954
1,6.281691,5.123892
2,6.913866,5.313473
3,5.384824,4.649200
4,6.304187,5.292786
...,...,...
292,4.246918,4.484358
293,4.246918,4.484358
294,4.658370,4.541445
295,5.923428,5.981714


In [ ]:
df_potency_prediction.to_csv('final_potency_predictions.csv', index=False)